# Comparison of VCH under different translations

This notebook compares the volume of the convex hull (VCH) of structures from `ba10` and `nmd18` data set
under different translations. Given a set of coordinates, VCH is invariant to rigid translations, such as tranlations, rotation and reflection. We use ASE for translating structures. 

The data can be download from https://qmml.org/datasets.html

In [1]:
from ase import Atoms
from scipy.spatial import ConvexHull
from tqdm.notebook import tqdm
import numpy as np

In [2]:
def read_file(fname):
    with open(fname) as fin:
        lines = filter(None, (line.rstrip() for line in fin))
    
        structures = []
        for structure in tqdm(read_next_structure(lines)):
            structures.append(structure)
            
    return(structures)

In [3]:
def read_next_structure(lines):
    
    while(lines):
    
        try:
            natoms =  int(next(lines))
        except:
            return
        info = next(lines).split()
        if len(info) == 4:
            (name, method, p1, p2) = info
        else:
            (p1, p2, p3) = info
        positions = []
        symbols = []
        for atom in range(natoms):
            (symbol, x, y, z) = next(lines).split()
            symbols.append(symbol)
            positions.append([float(x), float(y), float(z)])
        cell = []
        for axis in range(3):
            (x, y, z) = next(lines).split()
            cell.append([float(x), float(y), float(z)])
    
        structure = Atoms(symbols = symbols,
          positions = positions,
          cell = cell,
          pbc=True)
        
        yield(structure)

In [7]:
def get_volume(coordinates):
    try: #is the system plannar?
        volume = ConvexHull(coordinates).volume
    except:
        volume = 0
    return volume
    

In [9]:
max_diff = 0

structures = read_file("ba10-18.xyz")

for structure in tqdm(structures):
    volume = get_volume(structure.get_positions())
    for p in range(10):
        structure.translate(np.random.random(3)*10)
        new_volume = get_volume(structure.get_positions())
        if abs(new_volume - volume) > max_diff:
            max_diff = abs(new_volume - volume)
            
print("The maximum difference in VCH is: ", max_diff)

0it [00:00, ?it/s]

  0%|          | 0/15950 [00:00<?, ?it/s]

The maximum difference in VCH is:  2.6645352591003757e-13


In [10]:
max_diff = 0

structures = read_file("nmd18r.xyz")

for structure in tqdm(structures):
    volume = get_volume(structure.get_positions())
    for p in range(10):
        structure.translate(np.random.random(3)*10)
        new_volume = get_volume(structure.get_positions())
        if abs(new_volume - volume) > max_diff:
            max_diff = abs(new_volume - volume)
    
print("The maximum difference in VCH is: ", max_diff)

0it [00:00, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

The maximum difference in VCH is:  2.2737367544323206e-12
